# STEP : Report

Using QIIME2 aggregate vizualizations to create a report of the analysis.


## Methods
[qiime tools make-report](https://github.com/qiime2/q2cli/blob/18899635be6a56e7fec939201c3cf1c0b4afe49c/q2cli/builtin/tools.py#L557)
```
qiime tools make-report \
  alpha-rarefaction.qzv \
  beta-rarefaction-braycurtis-upgma-spearman-1000.qzv \
  --result-path report.qzv
```


## Setup and settings

In [ ]:
# Importing packages
import os
import pandas as pd
from qiime2 import Artifact
from qiime2 import Visualization
from qiime2 import Metadata
from qiime2.plugins.diversity.visualizers import alpha_rarefaction
from qiime2.plugins.diversity.visualizers import beta_rarefaction

from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.feature_table.methods import filter_seqs

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# from qiime2.plugins import demux, deblur, quality_filter, \
#                            metadata, feature_table, alignment, \
#                            phylogeny, diversity, emperor, feature_classifier, \
#                            taxa, composition

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [ ]:
metadata_file = '/home/lauro/nupeb/rede-micro/redemicro-miliane-nutri/data/raw/metadata/miliane-metadata-CxAC.tsv'
base_dir = os.path.join('/', 'home', 'lauro', 'nupeb', 'rede-micro', 'redemicro-miliane-nutri')
experiment_name = 'miliane-CxAC-trim'
replace_files = False

In [ ]:
# Parameters
experiment_name = "yuri"
base_dir = "/home/lauro/yuri/project"
manifest_file = "/home/lauro/yuri/initial_data/manifest.csv"
metadata_file = "/home/lauro/yuri/initial_data/metadata.tsv"
class_col = "sample-collection"
classifier_file = "/home/lauro/qiime2-classifiers/silva-138-99-nb-classifier.qza"
replace_files = False
phred = 20
trunc_f = 0
trunc_r = 0
overlap = 12
threads = 6
trim = {"overlap": 8, "forward_primer": "CCTACGGGRSGCAGCAG", "reverse_primer": "GGACTACHVGGGTWTCTAAT"}


### Getting all qiime2 artifacts paths

In [ ]:
file_map = {
    'demux-paired': 'Raw Sequences',
    'demux-paired-trim': 'Trimmed Raw Sequences',
    'dada2-stat': 'DADA2 Statistics',
    'dada2-tabs': 'DADA2 Feature Table',
    'dada2-reps': 'DADA2 Representative Sequences',
    'dada2-core': 'DADA2 Core Metrics',
    'alpha-rarefaction': 'Alpha Rarefaction',
    'beta-rarefaction': 'Beta Rarefaction',
}

In [ ]:
tmp_dir = os.path.join(base_dir, 'experiments', experiment_name, 'tmp')
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

In [ ]:
qiime_folder = os.path.join(base_dir, 'experiments', experiment_name, 'qiime-artifacts')

# List all .qzv files in the qiime_folder
qiime_views = [ f for f in os.listdir(qiime_folder) if f.endswith('.qzv') ]

report_path = os.path.join(qiime_folder, 'report')

In [ ]:
org_views = { k: [] for k, v in file_map.items() }
new_views = { k: [] for k, v in file_map.items() }

for view in qiime_views:
    new_path = os.path.join(tmp_dir, view)
    key = view.split('.')[0]

    if key in file_map:
        tmp = file_map[key].replace(' ', '_')
        new_name = tmp + '.qzv'
        new_path = os.path.join(tmp_dir, new_name)
        new_views[key].append(new_path)
        org_views[key].append(os.path.join(qiime_folder, view))

    else:
        for key2 in file_map.keys():
            k = key2 + '-'
            if k in key:
                tmp = file_map[key2].replace(' ', '_')
                new_name = key.replace(k, tmp + '_') + '.qzv'
                new_path = os.path.join(tmp_dir, new_name)
                new_views[key2].append(new_path)
                org_views[key2].append(os.path.join(qiime_folder, view))
                break
new_views

In [ ]:
org_views

In [ ]:
# Copying files to the new paths
for key, paths in new_views.items():
    for i, path in enumerate(paths):
        org_path = org_views[key][i]
        cmd = f'cp "{org_path}" "{path}"'
        print(cmd)
        os.system(f'cp "{org_path}" "{path}"')

In [ ]:
# Create make report command
make_report_cmd = 'qiime tools make-report'
for key, paths in new_views.items():
    for path in paths:
        make_report_cmd += f' "{path}"'
make_report_cmd += f' --report-path "{report_path}.qzv"'
print(make_report_cmd)

In [ ]:
# Executing the command
try:
    os.system(make_report_cmd)
except Exception as e:
    print(f'Error creating report: {e}')
